In [1]:
from utils.seed_setter import set_seed
set_seed()

In [2]:
import pickle,os

with open(os.path.join('./pickle_data/train_test_data/test_data.pickle'), 'rb') as f:
    x_test, y_test = pickle.load(f)
f.close()

In [3]:
from utils.black_box import BlackBox
black_box = BlackBox()

In [4]:
all_preds = black_box.predict_all(x_test)

In [5]:
adversarial_list = []

attack_list = []

for sent, lab, pred in list(zip(x_test, y_test, all_preds)):
        if round(pred) == lab and len(sent) < 500:
            attack_list += [(sent,lab)]
        elif round(pred) != lab and len(sent) < 500:
            adversarial_list += [(sent,lab)]

In [6]:
len(attack_list)

1610

In [7]:
len(adversarial_list)

144

In [8]:
attack_list[:3]

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailers get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the book is better..",
  0),
 ('The message of this movie is "personality is more important than beauty". Jeanine Garofalo is supposed to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will Hollywood stop with this hypocrisy?In my opinion, despite the m

In [9]:
print('Accuracy for the attack list:')
black_box.evaluate([sent for sent, lab in attack_list], [lab for sent, lab in attack_list])

Accuracy for the attack list:
1610/1610 [==============================] - 4s 3ms/sample - loss: 0.0466 - acc: 1.0000


[0.0465955188139255, 1.0]

In [10]:
print('Accuracy for the adversarial list:')
black_box.evaluate([sent for sent, lab in adversarial_list], [lab for sent, lab in adversarial_list])

Accuracy for the adversarial list:
144/144 [==============================] - 0s 3ms/sample - loss: 2.2604 - acc: 0.0000e+00


[2.2603875266181097, 0.0]

In [11]:
#del black_box

In [12]:
'''def most_similar(word, delta = 0.5, num_words = 20):
    
    try:
        index = tokens_dictionary[word]
    except:
        return []
    
    if (index > distance_matrix.shape[0]):
        return []
    
    dist_order = np.argsort(distance_matrix[index,:])[1:num_words+1]
    dist_list = distance_matrix[index][dist_order]
    
    mask = np.ones_like(dist_list)
    mask = np.where(dist_list < delta)
    return [inverse_tokens_dictionary[index] for index in dist_order[mask]]'''

'def most_similar(word, delta = 0.5, num_words = 20):\n    \n    try:\n        index = tokens_dictionary[word]\n    except:\n        return []\n    \n    if (index > distance_matrix.shape[0]):\n        return []\n    \n    dist_order = np.argsort(distance_matrix[index,:])[1:num_words+1]\n    dist_list = distance_matrix[index][dist_order]\n    \n    mask = np.ones_like(dist_list)\n    mask = np.where(dist_list < delta)\n    return [inverse_tokens_dictionary[index] for index in dist_order[mask]]'

In [13]:
'''import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox
import time


class Attacker(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 8 #20
        self.__top_lm = 4
        self.__max_gens = 15
        self.__max_children = 4
        self.__max_pop_members = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        #perturbation_start_time = time.time()
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : min(self.__top_neighbours, len(neighbours))], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[- min(self.__top_lm, len(lm_preds)) : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_sentences, score, changed_words_list + [word_index])]
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        #print('%perturbation_time = {} seconds\n'.format(int(time.time() - perturbation_start_time)))
        #print('Final review:')
        
        if y_target == 0:
            #print(self.__rejoin_review(adv_reviews_sorted[0][0]) + '\n')
            #print('Score:' + str(adv_reviews_sorted[0][1]) + '\n')
            return adv_reviews_sorted[0]
            
        else:
            #print(self.__rejoin_review(adv_reviews_sorted[-1][0]) + '\n')
            #print('Score:' + str(adv_reviews_sorted[-1][1]) + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        parent1_copy = parent1[0]
        parent2_copy = parent2[0]
        changed_word_list = []
        for i in range(len(parent1_copy)):
            for j in range(len(parent1_copy[i])):
                if np.random.uniform() < 0.5:
                    parent1_copy[i][j] = parent2_copy[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        return parent1_copy, changed_word_list
    
    def __get_neighbours_dictionary(self, sentences):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )

        return neighbours_dictionary
    
    def __get_words_to_change(self, neighbours_dictionary, sentences, changed_word_list = []):
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        if length_sum == 0:
            return None
        
        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = min(len(np.nonzero(probabilities)[0]), self.__max_words)
        
        return np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
    
        
    def attack (self, x_orig, y_orig):
        
        attack_start_time = time.time()
        
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        
        neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
        words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
        if words_to_change is None:
            return None
        
        print('Original sentence: \n' + x_orig + '\n')
        
        print('Generating population... \n')
        
        generation_start_time = time.time()

        population = []
        for index in words_to_change:
            #print('Starting perturbation {}/{}...\n'.format(i + 1, len(words_to_change)))
            new_member = self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            )
            
            if round(new_member[1]) == y_target:
                print('ATTACK SUCCESS!!!\n')
                final_sentence = self.__rejoin_review(new_member[0])
                print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                    self.__rejoin_review(new_member[0]), new_member[1]
                ))
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return final_sentence
            
            population += [new_member]

        for i in range(self.__max_gens):

            if len(population) == 0:
                print('ATTACK FAILED...\n')
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return None
            
            print('Generation #{}: \n'.format(i+1))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    


            if y_target == 0:
                best_attack = sorted_population[0]
                sorted_population = sorted_population[ : min(self.__max_pop_members, len(sorted_population))]

            else:
                best_attack = sorted_population[-1]
                sorted_population = sorted_population[- min(self.__max_pop_members, len(sorted_population)) : ]
                
            #print(len(sorted_population))
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(self.__rejoin_review(best_attack[0]), best_attack[1]))

            #if round(best_attack[1]) == y_target:
            #    return self.__rejoin_review(best_attack[0])
            
            print('%generation_time = {} seconds\n'.format(int(time.time() - generation_start_time)))
        
            best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score, _ in sorted_population])
            
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(self.__max_children)]
            
            perturbated_children = []
            
            print('Regenerating population... \n')
            
            generation_start_time = time.time()
            
            #i = 0
            
            for child in children:
                
                sentences = child[0]
                
                neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
                words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences, child[1])
                
                if words_to_change is None:
                    words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
                if words_to_change is None:
                    continue

                for index in words_to_change:
                    #print('Starting perturbation {}/{}:\n'.format(i + 1, len(children) * len(words_to_change)))
                    #i = i+1
                    new_member = self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    )
                    
                    
                    
                    if round(new_member[1]) == y_target:
                        print('ATTACK SUCCESS!!!\n')
                        final_sentence = self.__rejoin_review(new_member[0])
                        print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                            self.__rejoin_review(new_member[0]), new_member[1]
                        ))
                        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                        return final_sentence

                    perturbated_children += [new_member]

            population = [best_attack] + perturbated_children
        
        print('ATTACK FAILED...\n')
        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
        return None'''

'import string\nimport re\nimport os\nimport numpy as np\nimport pickle\nimport licensed_scripts.lm_1b_eval as google_language_model_utils\nimport nltk\nfrom utils.black_box import BlackBox\nimport time\n\n\nclass Attacker(object):\n    \n    def __init__ (self):\n        with open(os.path.join(\'./pickle_data/attack_utils/tokens_dictionary.pickle\'), \'rb\') as f:\n            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)\n            self.__tokens_dictionary = tokens_dictionary\n            self.__inverse_tokens_dictionary = inverse_tokens_dictionary\n        f.close()\n        \n        self.__black_box = BlackBox()\n        self.__distance_matrix = np.load(os.path.join(\'./numpy_files/distance_matrix.npy\'))\n        self.__google_lm = google_language_model_utils.LM()\n        self.__stopwords = nltk.corpus.stopwords.words(\'english\')\n        self.__sentence_tokenizer = nltk.data.load(\'tokenizers/punkt/english.pickle\')\n        self.__tree_bank_word_tokenizer = 

In [14]:
import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox
import time


class Attacker(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 8 #20
        self.__top_lm = 4
        self.__max_gens = 15
        self.__max_children = 2
        self.__max_pop_members = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        #perturbation_start_time = time.time()
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : min(self.__top_neighbours, len(neighbours))], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[- min(self.__top_lm, len(lm_preds)) : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_sentences, score, changed_words_list + [word_index])]
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        #print('%perturbation_time = {} seconds\n'.format(int(time.time() - perturbation_start_time)))
        #print('Final review:')
        
        if y_target == 0:
            #print(self.__rejoin_review(adv_reviews_sorted[0][0]) + '\n')
            #print('Score:' + str(adv_reviews_sorted[0][1]) + '\n')
            return adv_reviews_sorted[0]
            
        else:
            #print(self.__rejoin_review(adv_reviews_sorted[-1][0]) + '\n')
            #print('Score:' + str(adv_reviews_sorted[-1][1]) + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        parent1_copy = parent1[0]
        parent2_copy = parent2[0]
        changed_word_list = []
        for i in range(len(parent1_copy)):
            for j in range(len(parent1_copy[i])):
                if np.random.uniform() < 0.5:
                    parent1_copy[i][j] = parent2_copy[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        score = self.__black_box.predict_sentiment(self.__rejoin_review(parent1_copy))
        return parent1_copy, score, changed_word_list
    
    def __get_neighbours_dictionary(self, sentences):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )

        return neighbours_dictionary
    
    def __get_words_to_change(self, neighbours_dictionary, sentences, changed_word_list = []):
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        if length_sum == 0:
            return None
        
        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = min(len(np.nonzero(probabilities)[0]), self.__max_words)
        
        return np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
    
        
    def attack (self, x_orig, y_orig):
        
        attack_start_time = time.time()
        
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        
        neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
        words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
        if words_to_change is None:
            return None
        
        print('Original sentence: \n' + x_orig + '\n')
        
        print('Generating population... \n')
        
        generation_start_time = time.time()

        population = []
        for index in words_to_change:
            #print('Starting perturbation {}/{}...\n'.format(i + 1, len(words_to_change)))
            new_member = self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            )
            
            if round(new_member[1]) == y_target:
                print('ATTACK SUCCESS!!!\n')
                final_sentence = self.__rejoin_review(new_member[0])
                print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                    self.__rejoin_review(new_member[0]), new_member[1]
                ))
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return final_sentence
            
            population += [new_member]

        for i in range(self.__max_gens):

            if len(population) == 0:
                print('ATTACK FAILED...\n')
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return None
            
            print('Generation #{}: \n'.format(i+1))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    


            if y_target == 0:
                best_attack = sorted_population[0]
                sorted_population = sorted_population[ : min(self.__max_pop_members, len(sorted_population))]

            else:
                best_attack = sorted_population[-1]
                sorted_population = sorted_population[- min(self.__max_pop_members, len(sorted_population)) : ]
                
            #print(len(sorted_population))
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(self.__rejoin_review(best_attack[0]), best_attack[1]))

            #if round(best_attack[1]) == y_target:
            #    return self.__rejoin_review(best_attack[0])
            
            print('%generation_time = {} seconds\n'.format(int(time.time() - generation_start_time)))
        
            #best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score, _ in sorted_population])
            
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=len(sorted_population), p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=len(sorted_population), p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(len(sorted_population))]
            
            sorted_children =  sorted(children, key=lambda x: x[1])    
            
            if y_target == 0:
                #best_attack = sorted_children[0]
                sorted_children = sorted_children[ : min(self.__max_children, len(sorted_children))]

            else:
                #best_attack = sorted_children[-1]
                sorted_children = sorted_children[- min(self.__max_children, len(sorted_children)) : ]
            
            perturbated_children = []
            
            print('Regenerating population... \n')
            
            generation_start_time = time.time()
            
            for child in sorted_children:
                
                sentences = child[0]
                
                neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
                words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences, child[2])
                
                if words_to_change is None:
                    words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
                if words_to_change is None:
                    continue

                for index in words_to_change:
                    #print('Starting perturbation {}/{}:\n'.format(i + 1, len(children) * len(words_to_change)))
                    #i = i+1
                    new_member = self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    )
                    
                    
                    
                    if round(new_member[1]) == y_target:
                        print('ATTACK SUCCESS!!!\n')
                        final_sentence = self.__rejoin_review(new_member[0])
                        print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                            self.__rejoin_review(new_member[0]), new_member[1]
                        ))
                        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                        return final_sentence

                    perturbated_children += [new_member]

            population = [best_attack] + perturbated_children
        
        print('ATTACK FAILED...\n')
        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
        return None

In [15]:
attacker = Attacker()

LM vocab loading done


Recovering graph.


INFO:tensorflow:Recovering Graph google_language_model\graph-2016-09-10.pbtxt


Recovering checkpoint google_language_model\ckpt-*


In [16]:
final_list = []

BATCH_SIZE = 25

for i in range(BATCH_SIZE):
    print('####################ATTACK {}/{}:####################'.format(i+1, BATCH_SIZE))
    new_review = attacker.attack(attack_list[i][0], attack_list[i][1])
    if new_review is None:
        new_review = attack_list[i][0]
    final_list += [(new_review, attack_list[i][1])]

####################ATTACK 1/25:####################
Original sentence: 
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Generating population... 

Generation #1: 

Best Adversarial:
 This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one. 
 Score: 0.11358830332756042

%generation_time = 166 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog 

Regenerating population... 

Generation #2: 

Best Adversarial:
 Worst heaps of drivel to today! Everyone involved with this productive should be ashamed of themselves. Not one single element of the movie was anything slightly like an original idea. A first grader telling you a story about nap time is more entertaining. 
 Score: 0.0014815093018114567

%generation_time = 297 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 Worst pile of mania to today! Everyone involved with this productive should be ashamed of themselves. Not one single element of the movie was anything slightly love an original idea. A premiere grader telling you a story about nap time is more entertaining. 
 Score: 0.008928148075938225

%generation_time = 297 seconds

Regenerating population... 

ATTACK SUCCESS!!!

Final adversarial sentence:
 harshest pile of mania to today! Everyone involved with this productive should be ashamed of themselves. Not one single element of the movie was anythi

Regenerating population... 

Generation #10: 

Best Adversarial:
 What more can you ask for? A great screenplay base on one of the purest plays of the latter half of the 20th century, two fine emotional performances by Courtney and Finney, a realistic vision of war time birmingham, a grand supporting blown. This film takes you on an mentally rollercoaster through comedy, regret, losses and fulfillment. if you are in the cinema it is even more efficiency. This is a true 10 on the ratings scale! 
 Score: 0.6580274701118469

%generation_time = 294 seconds

Regenerating population... 

Generation #11: 

Best Adversarial:
 What more can you ask for? A great screenplay base on one of the purest plays of the latter half of the 20th century, two fine emotional performances by Courtney and Finney, a realistic vision of war time birmingham, a grand supporting blown. This film takes you on an mentally rollercoaster through comedy, regret, losses and fulfillment. if you are in the cinema it is eve

Regenerating population... 

Generation #2: 

Best Adversarial:
 This is the very La Nouvelle Vague. One of the best films of the New Wave and I dare say one of the first ten ever made! Why? The atmosphere, the story, the actors( actress) are all brilliant. This is the theater, a queer tale, the hayat, the film. Paris. Thank you Mr. Rivette. 
 Score: 0.994109034538269

%generation_time = 296 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 This is the very La Nouvelle blurry. One of the best movie of the New Wave and I dare assert one of the first ten ever made! Why? The atmosphere, the story, the actors( actress) are all brilliant. This is the theater, a queer tale, the life, the film. Paris. Thank you Mr. Rivette. 
 Score: 0.9656664133071899

%generation_time = 297 seconds

Regenerating population... 

Generation #4: 

Best Adversarial:
 This is the very La Nouvelle blurry. One of the higher movie of the New Wave and I dare assert one of the first ten ever ma

Regenerating population... 

Generation #8: 

Best Adversarial:
 My initially Fassbinder was a magnificent experience. filmmaking and alternative film( small hall, with embarrassing seats; public had to wait while filmrolls were changed) were perfected match.There were many cliches used in the film, but Fassbinder presented them so cleverly that I found them really amusing. Sound was also marvellous( sometimes back being louder than dialogue).Everything sounded to be in right place. And I adore the way how after- karas- period was presented. Real droll! 
 Score: 0.5608624815940857

%generation_time = 296 seconds

Regenerating population... 

Generation #9: 

Best Adversarial:
 My initially Fassbinder was a magnificent experience. filmmaking and alternative film( small hall, with embarrassing seats; public had to wait while filmrolls were changed) were perfected match.There were many cliches used in the film, but Fassbinder presented them so cleverly that I found them really amusing. So

...

Generation #1: 

Best Adversarial:
 I am a fan of the Nightmare series but this one is horrible. The deaths are so trendy. If you were to watch this 20 years later the whole nintendo scene is outdated. I did like the flashbacks. I think they should have just made a prequel about when he was still alive. That would have been more thrilling. This is a movie you can take or leave. Depends on how much spare time you have. 
 Score: 0.011744058690965176

%generation_time = 148 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 I am a fan of the Nightmare series but this one is horrible. The deaths are so trendy. If you were to watch this 20 years later the overall nintendo scene is outdated. I did like the flashbacks. I think they should have just made a prequel about when he was still alive. That would have been more thrilling. This is a cinematic you can take or leave. Depends on how much spare time you have. 
 Score: 0.020684463903307915

%generation_time = 295 seco

Regenerating population... 

Generation #3: 

Best Adversarial:
 What can I say? I know this movie from start to complete. It's comic. It's an forte link to my past and will change the way I opinion film in the future. Hypothetically speaking:) The down-fall? There's no Socrates Johnson! 
 Score: 0.581102728843689

%generation_time = 297 seconds

Regenerating population... 

Generation #4: 

Best Adversarial:
 What can I say? I know this movie from launch to complete. It's comic. It's an forte link to my past and will change the way I opinion film in the future. Hypothetically speaking:) The down-fall? There's no Socrates Johnson! 
 Score: 0.5060469508171082

%generation_time = 298 seconds

Regenerating population... 

ATTACK SUCCESS!!!

Final adversarial sentence:
 What can I say? I realise this movie from launch to complete. It's comic. It's an forte link to my past and will change the way I opinion film in the future. Hypothetically speaking:) The down-fall? There's no Socrates John

Regenerating population... 

Generation #3: 

Best Adversarial:
 After watching John preform this one of a kind show, I had to share.....It was really something to watch a cultured hombre portray himself as a child. I like the fact that with every character he`` became,`` you could picture what they looked like. It is more entertaining when you can understand the individual. `` Freak`` is what factual`` stand up`` should be. John is REAL gifted. 
 Score: 0.692997932434082

%generation_time = 297 seconds

Regenerating population... 

ATTACK SUCCESS!!!

Final adversarial sentence:
 After watching John preform this someone of a kind show, I had to share.....It was really something to watch a cultured hombre portray himself as a child. I like the fact that with every character he`` became,`` you could picture what they looked like. It is more entertaining when you can understand the individual. `` Freak`` is what factual`` stand up`` should be. John is REAL gifted. 
 Score: 0.4996873438358

Regenerating population... 

ATTACK SUCCESS!!!

Final adversarial sentence:
 This is my favorite display. I thinks it is wholly brilliant. thanking to David Chase for make this into my hayat. Season 11. The Sopranos: 5/52. 46 prolonged: 4.5/53. Denial, outrage, receiving: 5/54. Meadowlands: 4/55. College: 6. Pax Soprana: 5/57. Down Neck: 4.5/58. The Legend of Tennessee Moltisanti: 5/59. Boca: 4.5/510. A struck Is a struck: 3.5/511. someone Knows nothing: 5/512. Isabella: 5/513. I Dream of deanna Cusamano: 5/5 
 Score: 0.3744359314441681

%total_time = 1912 seconds


####################ATTACK 24/25:####################
Original sentence: 
Makes "Invasion USA" look like "Apocalypse Now". This one can only be recommended to US skinheads, John Birch supporters or militia members. The message is very simple : let's shoot them all - gooks, commies, latinos, everything that isn't American.Besides, this is badly acted, badly scripted, badly interpreted, incredibly stupid but no fun at all. Th

Generation #1: 

Best Adversarial:
 This film very succeed in the Film festival in Karlovy Vary, Czech rep. Logically. It' s basis on a very good book, excellent actors, good camera, the best director and ICELANDIC. It was probably the most black comedy I've seen. I really DO recommend it to you. 
 Score: 0.9365444183349609

%generation_time = 149 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 This filmmaking very succeed in the Film festival in Karlovy Vary, Czech rep. Logically. It' s based on a very good book, excellent actors, decent camera, the best director and ICELANDIC. It was probably the most negro comedy I've seen. I really DO recommend it to you. 
 Score: 0.7837022542953491

%generation_time = 298 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 This filmmaking very succeed in the Film festival in Karlovy Vary, Czech rep. soundly. It' s based on a very good book, excellent actors, good camera, the best director and ICELAND

In [17]:
print(final_list)

[('This movies could cure sleep disorders, dont how bad it is. The story dragged, and the naughty guy is not that scary. You will not even see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one.', 0), ("The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be honest, the book is enhanced..", 0), ("The messenger of this cinematic is`` personality is more important than beauty``. arlene Garofalo is supposed to be the`` ugly duckling'', but the funny aspect is that she's not at all ugly( actually she's a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this cinema work if the`` ugly duckling'' was really repulsive? When will Hollywood stop with this hypocrisy? In my views

In [18]:
black_box.evaluate([sent for sent, lab in final_list], [lab for sent, lab in final_list])

25/25 [==============================] - 0s 8ms/sample - loss: 0.9110 - acc: 0.0000e+00


[0.910988450050354, 0.0]

In [19]:
######################Bigger size#####################

In [13]:
'''import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox

class Attack(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 8 #20
        self.__top_lm = 4
        self.__max_gens = 10
        self.__max_children = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        print('Starting perturbation:')
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        #if self.__top_neighbours > len(neighbours):
        #    top_neighbours = len(neighbours)
        #else:
        #    top_neighbours = self.__top_neighbours
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : min(self.__top_neighbours, len(neighbours))], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[- min(self.__top_lm, len(lm_preds)) : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_sentences, score, changed_words_list + [word_index])]
            #print(adv_review)
            #print('Score:' + str(score) + '\n')
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        print('Final review:')
        
        if y_target == 0:
            print(self.__rejoin_review(adv_reviews_sorted[0][0]) + '\n')
            print('Score:' + str(adv_reviews_sorted[0][1]) + '\n')
            return adv_reviews_sorted[0]
            
        else:
            print(self.__rejoin_review(adv_reviews_sorted[-1][0]) + '\n')
            print('Score:' + str(adv_reviews_sorted[-1][1]) + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        parent1_copy = parent1[0]
        parent2_copy = parent2[0]
        changed_word_list = []
        for i in range(len(parent1_copy)):
            for j in range(len(parent1_copy[i])):
                if np.random.uniform() < 0.5:
                    parent1_copy[i][j] = parent2_copy[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        #print(splitted_parent1)
        return parent1_copy, changed_word_list
    
    def __get_neighbours_dictionary(self, sentences):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )

        return neighbours_dictionary
    
    def __get_words_to_change(self, neighbours_dictionary, sentences, changed_word_list = []):
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = min(len(np.nonzero(probabilities)[0]), self.__max_words)#len(np.nonzero(probabilities)[0]) if len(np.nonzero(probabilities)[0]) < self.__max_words else self.__max_words

        return np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
    
        
    def attack (self, x_orig, y_orig):
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        
        neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
        words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
        #print (words_to_change)
        
        print('Original sentence: \n' + x_orig + '\n')
        
        print('Generating population... \n')

        population = [self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            ) for index in words_to_change]
            
        for i in range(self.__max_gens):

            if len(population) == 0:
                return None
            
            print('Generation #{}: \n'.format(i))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    


            if y_target == 0:
                best_attack = sorted_population[0]
                sorted_population = sorted_population[ : min(5, len(sorted_population))]

            else:
                best_attack = sorted_population[-1]
                sorted_population = sorted_population[- min(5, len(sorted_population)) : ]
                
            print(len(sorted_population))
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(self.__rejoin_review(best_attack[0]), best_attack[1]))

            if round(best_attack[1]) == y_target:
                return self.__rejoin_review(best_attack[0])
        
            best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score, _ in sorted_population])
            
            #print('Pop scores: ' + str(pop_scores))
            
            #logits = np.exp(pop_scores - np.max(pop_scores))
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
            
            #print('Probab: ')
            #print(selection_probabilities)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                #print('New probab:')
                #print(selection_probabilities)
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(self.__max_children)]
            
            perturbated_children = []
            
            print('########################################\n\n')
            print('Regenerating population \n')
            
            for child in children:
                
                ######################################DUPLICATED CODE##############################################
                sentences = child[0]
                
                neighbours_dictionary = self.__get_neighbours_dictionary(sentences)
                words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences, child[1])
        
                ######################################END DUPLICATED CODE##############################################
                if (len(words_to_change)):
                    perturbated_children += [self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    ) for index in words_to_change]
            
            population = [best_attack] + perturbated_children
            
        return None'''

In [14]:
'''import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox

class Attack(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 8 #20
        self.__top_lm = 4
        self.__max_gens = 10
        self.__max_children = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        print('Starting perturbation:')
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        if self.__top_neighbours > len(neighbours):
            top_neighbours = len(neighbours)
        else:
            top_neighbours = self.__top_neighbours
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : top_neighbours - 1], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[-self.__top_lm : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_review, score, changed_words_list + [word_index])]
            #print(adv_review)
            #print('Score:' + str(score) + '\n')
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        print('Final review:')
        
        if y_target == 0:
            print(adv_reviews_sorted[0][0] + '\n')
            print('Score:' + str(adv_reviews_sorted[0][1]) + '\n')
            return adv_reviews_sorted[0]
            
        else:
            print(adv_reviews_sorted[-1][0] + '\n')
            print('Score:' + str(adv_reviews_sorted[-1][1]) + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        splitted_parent1 = self.__split_review(parent1[0])
        splitted_parent2 = self.__split_review(parent2[0])
        changed_word_list = []
        for i in range(len(splitted_parent1)):
            for j in range(len(splitted_parent1[i])):
                if np.random.uniform() < 0.5:
                    splitted_parent1[i][j] = splitted_parent2[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        #print(splitted_parent1)
        return self.__rejoin_review(splitted_parent1), changed_word_list
    
    def __get_neighbours_dictionary_and_words_to_change(self, sentences, changed_word_list = []):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )

        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = len(np.nonzero(probabilities)[0]) if len(np.nonzero(probabilities)[0]) < self.__max_words else self.__max_words

        words_to_change = np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
        return neighbours_dictionary, words_to_change

    
        
    def attack (self, x_orig, y_orig):
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        
        neighbours_dictionary, words_to_change = self.__get_neighbours_dictionary_and_words_to_change(sentences)
        
        #print (words_to_change)
        
        print('Original sentence: \n' + x_orig + '\n')
        
        print('Generating population... \n')

        population = [self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            ) for index in words_to_change]
            
        for i in range(self.__max_gens):

            if len(population) == 0:
                return None
            
            print('Generation #{}: \n'.format(i))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    

            #print('Final review:')

            if y_target == 0:
                best_attack = sorted_population[0]
            else:
                best_attack = sorted_population[-1]
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(best_attack[0], best_attack[1]))

            if round(best_attack[1]) == y_target:
                return best_attack[0]
        
            best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score, _ in sorted_population])
            
            #print('Pop scores: ' + str(pop_scores))
            
            #logits = np.exp(pop_scores - np.max(pop_scores))
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
            
            #print('Probab: ')
            #print(selection_probabilities)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                #print('New probab:')
                #print(selection_probabilities)
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(self.__max_children)]
            
            perturbated_children = []
            
            print('########################################\n\n')
            print('Regenerating population \n')
            
            for child in children:
                
                ######################################DUPLICATED CODE##############################################
                sentences = self.__split_review(child[0])
                
                neighbours_dictionary, words_to_change = self.__get_neighbours_dictionary_and_words_to_change(sentences, child[1])
                
                ######################################END DUPLICATED CODE##############################################
                if (len(words_to_change)):
                    perturbated_children += [self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    ) for index in words_to_change]
            
            population = [best_attack] + perturbated_children
            
        return None'''

'import string\nimport re\nimport os\nimport numpy as np\nimport pickle\nimport licensed_scripts.lm_1b_eval as google_language_model_utils\nimport nltk\nfrom utils.black_box import BlackBox\n\nclass Attack(object):\n    \n    def __init__ (self):\n        with open(os.path.join(\'./pickle_data/attack_utils/tokens_dictionary.pickle\'), \'rb\') as f:\n            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)\n            self.__tokens_dictionary = tokens_dictionary\n            self.__inverse_tokens_dictionary = inverse_tokens_dictionary\n        f.close()\n        \n        self.__black_box = BlackBox()\n        self.__distance_matrix = np.load(os.path.join(\'./numpy_files/distance_matrix.npy\'))\n        self.__google_lm = google_language_model_utils.LM()\n        self.__stopwords = nltk.corpus.stopwords.words(\'english\')\n        self.__sentence_tokenizer = nltk.data.load(\'tokenizers/punkt/english.pickle\')\n        self.__tree_bank_word_tokenizer = nltk.tokenize.Tre

In [15]:
attack = Attack()

LM vocab loading done


Recovering graph.


INFO:tensorflow:Recovering Graph google_language_model\graph-2016-09-10.pbtxt


Recovering checkpoint google_language_model\ckpt-*


In [16]:
final_list = []
for i in range(5):
    print('########ATTACK {}/4##########:'.format(i))
    final_list += [(attack.attack(attack_list[i][0], attack_list[i][1]) , attack_list[i][1])]

########ATTACK 0/4##########:
Original sentence: 
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Generating population... 

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This cinematic made me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.06825272

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story pushed, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.11705203

Starting perturbation:
Final review:
This film could cure sleep d

Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that frightening. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.10649207

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choice. He work on a real dog with this one.

Score:0.10665911

Starting perturbation:
Final review:
This movies could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.19016926

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how nasty it is. The

Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not still see this one on TBS reruns. This film delivered me wonder about Chuck film selected. He work on a real dog with this one.

Score:0.37016085

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not ever see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.27287844

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that frightening. You will not still see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one.

Score:0.3662077

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how 

Final review:
The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good influences, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some best films, but i desired they had n't added this one to their resume's. To be honest, the book is better..

Score:0.22258131

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some best films, but i desired they had n't added this individual to their resume's. To be honest, the book is better..

Score:0.16924392

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some nice effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some best films, but i desired they had n

Final review:
The trailers gets you to the movie, but the movie just was n't worth my 8.50.. it has some good influences, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be honest, the book is better..

Score:0.10935741

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good influences, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this individual to their resume's. To be honest, the book is better..

Score:0.107234135

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good influences, but the storyline.. yech, i like tommy lee jones and will smith as partners, they have both done some good films, but i wish they had 

...

Final review:
greatest pile of drivel to today! Everyone involved with this production should be ashamed of themselves. Not one single element of the movie was anything somewhat loved an original idea. A premiere grader told you a story about nap days is more entertaining.

Score:0.08432566

Starting perturbation:
Final review:
greatest pile of drivel to yesterday! Everyone involved with this production should be ashamed of themselves. Not one single element of the movie was anything somewhat loved an original idea. A premiere grader told you a story about nap time is more entertaining.

Score:0.078622706

Starting perturbation:
Final review:
greatest pile of drivel to today! Everyone involved with this production should be ashamed of themselves. Not nobody single element of the movie was anything somewhat loved an original idea. A premiere grader told you a story about nap time is more entertaining.

Score:0.111783266

Starting perturbation:
Final review:
greatest pile of drivel to to

Final review:
I think that the movie was really better. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.85094506

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.79738826

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the filmmaker has succeeded in showing the status of women in ru

Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they endure at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.7723491

Starting perturbation:
Final review:
I think that the movie was basically good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.6763771

Starting perturbation:
Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has successful in showing the status of women in 

Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeed in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a few and has focussed more on the violence faced by her.

Score:0.6677433

Starting perturbation:
Final review:
I think that the movie was actually fine. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a few and has focussed more on the violence faced by her.

Score:0.7073949

Starting perturbation:
Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rura

Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in proving the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a few and has focussed more on the violence faced by her.

Score:0.65197474

Starting perturbation:
Final review:
I think that the movie was actually good. Subject, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's feature a few and has focussed more on the violence faced by her.

Score:0.7632739

Starting perturbation:
Final review:
I think that the movie was actually good. matter, acting and Nusrat Fateh ALi Khan's music were magnificent. Although the director has succeeded in showing the status of women in rura

In [17]:
print(final_list)

[('This film could cure sleep disorders, thats how nasty it is. The story dragged, and the naughty guy is not that scary. You will not still see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one.', 0), ("The trailers get you to the movie, but the movie just was n't worth my 8.50.. it has some good effects, but the storyline.. yech, i loved tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their revival's. To be honest, the pound is better..", 0), ("The message of this movie is`` personality is more important than beauty``. Jeanine Garofalo is supposed to be the`` ugly duckling'', but the hilarious thing is that she's not at all ugly( actually she's a lot more attractive than Uma Thurman, the friend who looks like a model).Now, must this movie job if the`` ugly duckling'' was really unattractive? When will Hollywood stop with this hypocrisy? In my opin

In [18]:
black_box.evaluate([sent for sent, lab in final_list], [lab for sent, lab in final_list])

5/5 [==============================] - 0s 20ms/sample - loss: 0.9254 - acc: 0.0000e+00


[0.925439715385437, 0.0]

In [ ]:
####################################################################################################################

In [97]:
attack = Attack()

LM vocab loading done


Recovering graph.


INFO:tensorflow:Recovering Graph google_language_model\graph-2016-09-10.pbtxt


Recovering checkpoint google_language_model\ckpt-*


In [98]:
#del attack

In [99]:
adv_x0 = attack.attack(attack_list[4][0], attack_list[4][1])

Original sentence: 
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Generating population... 

Starting perturbation:


KeyboardInterrupt: 

In [54]:
attack_list[4][0]

"I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her."

In [55]:
adv_x0

"I think that the movie was frankly good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of females in farmers areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her."

In [56]:
black_box.predict_sentiment(adv_x0)

0.40761864

In [57]:
black_box.predict_sentiment(attack_list[4][0])

0.8942218

In [58]:
attack_list[4][1]

1

In [100]:
final_list = []
for i in range(5):
    print('########ATTACK {}/4##########:'.format(i))
    final_list += [(attack.attack(attack_list[i][0], attack_list[i][1]) , attack_list[i][1])]

########ATTACK 0/4##########:
Original sentence: 
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Generating population... 

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.056767795

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this individual.

Score:0.046122003

Starting perturbation:
Final review:
This film could cu

Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.2132879

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This cinematic made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.09025106

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the naughty guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.12718247

Starting perturbation:
Final review:
This film could cure sleep disorders, thats 

Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He job on a real dog with this one.

Score:0.18484266

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story pushed, and the bad guy is not that terrifying. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.32526726

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that terrifying. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choice. He work on a real dog with this one.

Score:0.20358695

Starting perturbation:
Final review:
This film could treat sleep dis

Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be genuine, the book is better..

Score:0.109119184

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some best effects, but the narration.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be honest, the book is better..

Score:0.16926977

Starting perturbation:
Final review:
The trailers get you to the movie, but the cinema just was n't worth my 8. 50.. it has some best effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this

Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good cinema, but i wish they had n't added this one to their revival's. To be honest, the book is better..

Score:0.22389677

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i desired they had n't added this one to their revival's. To be honest, the book is better..

Score:0.23349345

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worthy my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added 

...

Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonely element of the movie was anything somewhat loved an original idea. A premiere grader telling you a saga about nap time is more entertaining.

Score:0.5180556

Starting perturbation:
Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonely aspect of the movie was anything somewhat loved an original idea. A premiere grader telling you a tale about nap time is more entertaining.

Score:0.7564107

Starting perturbation:
Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonesome element of the movie was anything somewhat loved an original idea. A premiere grader telling you a tale about nap time is more entertaining.

Score:0.553772

Starting perturbation:
Final review:
greatest pile of fancy to date! all invol

Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has succeeded in showing the status of women in peasant region and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8411289

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has succeeded in showing the status of women in rural region and how they suffering at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.88062495

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has avail in showing the status of women in rural region 

Final review:
I think that the movie was really good. matter, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's characters a bit and has focussed more on the violence faced by her.

Score:0.8563676

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they endure at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8385451

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural are

Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the chief has avail in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.90023226

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the chief has succeeded in displaying the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.89869654

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's musical were marvellous. Although the chief has succeeded in showing the status of women in rural areas and h

Final review:
I thinks that the movie was really good. topic, acting and Nusrat Fateh ALi Khan's musical were marvellous. albeit the director has succeeded in showing the status of female in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8198603

Starting perturbation:
Final review:
I believes that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's musical were marvellous. albeit the director has succeeded in showing the status of female in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.7958517

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's song were marvellous. albeit the director has succeeded in showing the status of female in rural ar

In [101]:
print(final_list)

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me amazed about Chuck cinematic choices. He work on a real dog with this one.', 0), ("The trailers gets you to the movie, but the cinema just was n't worth my 8. 50.. it has some best effects, but the storyline.. yech, i like tommy lee jones and will smith as gamers, they have both finished some good films, but i wish they had n't added this individual to their revival's. To be honest, the book is better..", 0), ("The message of this movie is`` personality is more important than beauty''. Jeanine Garofalo is supposed to be the`` ugly duckling'', but the funny thing is that she's not at all ugly( actually she's a multiple more exciting than Uma Thurman, the boyfriend who looks love a model). Now, would this movie work if the`` ugly duckling'' was really repulsive? When will Hollywood stop with this hypocr

In [102]:
black_box.evaluate([sent for sent, lab in final_list], [lab for sent, lab in final_list])

5/5 [==============================] - 0s 31ms/sample - loss: 1.0768 - acc: 0.0000e+00


[1.0767607688903809, 0.0]

In [2]:
################################################################################################################################

In [247]:
import re
import nltk.data
import nltk


def attack(review, y_target):
    stopwords = nltk.corpus.stopwords.words('english')
    sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sentence_tokenizer.tokenize(review)
    
    for sent_idx, sent in enumerate(sentences):
        if sent_idx < 3:
            splitted_text = sent.split()

            for w_idx, w in enumerate(splitted_text):
                if w_idx < 10:
                    w = w.lower()
                    pattern = r"[\w]+|[^\s\w]"
                    if len(re.findall(pattern, splitted_text[w_idx])) == 1 and w not in stopwords:
                        nearest_indexes = most_similar(w)
                        nearest_words = [inverse_tokens_dictionary[index] for index in nearest_indexes]
                        if len(nearest_words):
                            prefix = ' '.join(splitted_text[:w_idx])
                            print('Prefix:\n'+prefix)
                            suffix = ' '.join(splitted_text[w_idx + 1:])
                            print('Suffix:\n'+suffix)
                            lm_preds = google_language_model.get_words_probs(prefix, nearest_words, suffix)

                            score_list = []
                            for adv_w in np.argsort(lm_preds): 
                                adv_sentences = sentences[:]
                                adv_splitted_text = splitted_text[:]
                                adv_splitted_text[w_idx] = nearest_words[adv_w]
                                adv_sentences[sent_idx] = ' '.join(adv_splitted_text)
                                review_adv = ' '.join(adv_sentences)
                                score = black_box.predict_sentiment(review_adv)
                                score_list += [(adv_splitted_text, score)]
                                
                            adv_sentences_sorted =  sorted(score_list, key=lambda x: x[1])    
                            if y_target == 0:
                                splitted_text = adv_sentences_sorted[0][0]
                            else:
                                splitted_text = adv_sentences_sorted[-1][0]
                            sentences[sent_idx] = ' '.join(splitted_text)
    return ' '.join(sentences)

In [248]:
for i in range(5):
    x_orig = attack_list[i][0]
    y_orig = attack_list[i][1]
    
    y_target = int(not y_orig)
    
    adversarial_list.append((attack(x_orig, y_target), y_orig))
   
    
    
    print('Generated adversarial sentence')

Prefix:
This
Suffix:
could cure sleep disorders, thats how bad it is.
Prefix:
This movies
Suffix:
cure sleep disorders, thats how bad it is.
Prefix:
This movies did
Suffix:
sleep disorders, thats how bad it is.
Prefix:
This movies did remedy
Suffix:
disorders, thats how bad it is.
Prefix:
This movies did remedy hibernate disorders,
Suffix:
how bad it is.
Prefix:
This movies did remedy hibernate disorders, wouldnt how
Suffix:
it is.
Prefix:
The
Suffix:
dragged, and the bad guy is not that scary.
Prefix:
The tales dragged, and the
Suffix:
guy is not that scary.
Prefix:
The tales dragged, and the amiss
Suffix:
is not that scary.
Prefix:
You will not
Suffix:
see this one on TBS reruns.
Prefix:
You will not still
Suffix:
this one on TBS reruns.
Prefix:
You will not still admire this
Suffix:
on TBS reruns.
Generated adversarial sentence
Prefix:
The
Suffix:
get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jone

In [214]:
#sorted(score_list, key=lambda x: x[1])[0][0]

'Wow. I observed this movie and "Up" on the same day within an hour of each other at different theaters. I saw "Mr Bug" first, and was then totally disappointed in "Up"\'s follow-up. What a beautiful and touching film! Movies of the 1930s and 40s to us nowadays can be irking with their melodramatic acting and dialog, but as animation the same melodrama and groaning humor can be wonderful. And the soft "organic" lines of 30s drawing AND the music just puts you in a nice comfortable mood and you can enjoy the show with all its little characters: ladybugs, grasshoppers, bees, snails, stinkbugs, flies, mosquitoes, beetles, crickets, and more each with all their own cute little (but not overbearing) idiosyncrasies. The interaction with the human world, from nemesis (cigar smokers, high-heel wearers, innocent kick-the-can playing kids) to the kind-hearted, and to the unknown destroyers, is realistic and fascinating. You care for the bugs, AND Dick and Mary. The protagonist Hoppity is not som

In [ ]:
#preds = google_language_model.get_words_probs('hey', ['how', 'kill', 'stupid'], 'are you')

In [206]:
#[i for i in np.argsort(preds)]

[1, 2, 0]

In [188]:
#preds

array([1.1748647e-04, 1.0273188e-05, 4.0947703e-05], dtype=float32)

In [249]:
adversarial_list[-5:]

[('This movies did remedy hibernate disorders, wouldnt how naughty it is. The tales dragged, and the amiss fella is not that scary. You will not still admire this uno on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailer obtains you to the movie, but the cinema just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the ledger is better..",
  0),
 ('The voicemail of this cinema is "personality is more pivotal than beauty". janeane Garofalo is hypothetical to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will celebs halt with this hypocrisy?

In [254]:
attack_list[:5]

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailers get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the book is better..",
  0),
 ('The message of this movie is "personality is more important than beauty". Jeanine Garofalo is supposed to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will Hollywood stop with this hypocrisy?In my opinion, despite the m

In [251]:
black_box.predict_all([sent for sent, lab in adversarial_list[-5:]])

[0.8820844292640686,
 0.22996748983860016,
 0.0038647179026156664,
 0.7400735020637512,
 0.39983099699020386]

In [252]:
black_box.evaluate([sent for sent, lab in adversarial_list[-5:]], [lab for sent, lab in adversarial_list[-5:]])

5/5 [==============================] - 0s 17ms/sample - loss: 0.9334 - acc: 0.4000


[0.9334100484848022, 0.4]

In [253]:
black_box.evaluate([sent for sent, lab in attack_list[:5]], [lab for sent, lab in attack_list[:5]])

5/5 [==============================] - 0s 30ms/sample - loss: 0.0490 - acc: 1.0000


[0.04902750998735428, 1.0]